<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Keeping-Conda-up-to-date-and-installing-new-packages" data-toc-modified-id="Keeping-Conda-up-to-date-and-installing-new-packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Keeping Conda up to date and installing new packages</a></span><ul class="toc-item"><li><span><a href="#Installing-a-new-package-in-your-virtual-environment" data-toc-modified-id="Installing-a-new-package-in-your-virtual-environment-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Installing a new package in your virtual environment</a></span></li><li><span><a href="#Updating-an-existing-package" data-toc-modified-id="Updating-an-existing-package-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Updating an existing package</a></span></li></ul></li><li><span><a href="#Building-regression-models" data-toc-modified-id="Building-regression-models-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Building regression models</a></span><ul class="toc-item"><li><span><a href="#Visualizing-the-correlation-matrix" data-toc-modified-id="Visualizing-the-correlation-matrix-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Visualizing the correlation matrix</a></span></li><li><span><a href="#Visualizing-the-scatter-plot-matrix" data-toc-modified-id="Visualizing-the-scatter-plot-matrix-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Visualizing the scatter plot matrix</a></span></li></ul></li><li><span><a href="#Detour:-Visualization-of-the-regression-model" data-toc-modified-id="Detour:-Visualization-of-the-regression-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Detour: Visualization of the regression model</a></span></li><li><span><a href="#Checking-how-well-the-regression-model-works:-training-data" data-toc-modified-id="Checking-how-well-the-regression-model-works:-training-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Checking how well the regression model works: training data</a></span></li><li><span><a href="#Checking-how-well-the-regression-model-works:-testing-data" data-toc-modified-id="Checking-how-well-the-regression-model-works:-testing-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Checking how well the regression model works: testing data</a></span></li><li><span><a href="#Calculate-the-$R^2$-value" data-toc-modified-id="Calculate-the-$R^2$-value-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Calculate the $R^2$ value</a></span></li><li><span><a href="#Improving-the-model:-adding-extra-predictors" data-toc-modified-id="Improving-the-model:-adding-extra-predictors-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Improving the model: adding extra predictors</a></span></li><li><span><a href="#Challenge:-checking-the-MLR-model-on-the-testing-data" data-toc-modified-id="Challenge:-checking-the-MLR-model-on-the-testing-data-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Challenge: checking the MLR model on the testing data</a></span></li></ul></div>

> All content here is under a Creative Commons Attribution [CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/) and all source code is released under a [BSD-2 clause license](https://en.wikipedia.org/wiki/BSD_licenses).
>
>Please reuse, remix, revise, and [reshare this content](https://github.com/kgdunn/python-basic-notebooks) in any way, keeping this notice.

# Course overview

This is the fifth module of several (11, 12, 13, 14, 15 and 16), which refocuses the course material in the [prior 10  modules](https://github.com/kgdunn/python-basic-notebooks) in a slightly different way. It places more emphasis on

* dealing with data: importing, merging, filtering;
* calculations from the data;
* visualization of it.

In short: ***how to extract value from your data***.


# Module 15 Overview

In this module we will cover

* Visualization of a correlation matrix with a heat map
* Fitting a linear regression model to the data
* Visualization of the linear regression model
* Accessing data from your data frame using `.loc` and `.iloc`

**Requirements before starting**

* Have your Python installation working as you had for modules 11 to 14, including the Pandas library installed. 

* An extra requirement: install the `scikit-learn` and `seaborn` libraries. See instructions below.

## Keeping Conda up to date and installing new packages


Newer versions of packages are released frequently. You can update your packages (libraries), with these commands. Do this at the command line (not in Jupyter notebook!)::
```bash

    conda update -n base conda
    conda update --all
```

### Installing a new package in your virtual environment

You will come across people recommending different packages in Python for all sorts of interesting applications. For example, the library `seaborn` is often recommended for visualization. But you might not have it installed yet. 

This is how you can install the package called `seaborn` and `scikit-learn` packages in your virtual environment called ``myenv``:
```bash
    conda activate myenv    <--- change the last word in the command to the name of your actual environment
    conda install seaborn scikit-learn
```

Or in one line:
```bash
    conda install -n myenv seaborn scikit-learn
```


### Updating an existing package

Similar to the above, you can update a package to the latest version. Just change ``install`` to ``update`` instead.
Or in one line:
```bash
    conda update -n myenv seaborn scikit-learn
```

## Building regression models

In the [prior module](https://yint.org/pybasic14) you learned about setting the date and time when importing data, visualizing your data with box plots, histograms, line or time-series plots, and scatter plots. You applied these to your own data, and learned about the very powerful ``groupby`` function in Pandas.

In this module we will take these skills a step further, but first, we will learn about fitting regression models to some data. 

Start by importing Pandas, but also the tools to build regression models, from the `scikit-learn` library, which is imported as `sklearn`. You can read more about scikit-learn at their website: https://scikit-learn.org/stable/

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

We will use a data set that is concerns a [distillation column](https://openmv.net/info/distillation-tower), and predicting an important output variable, called the Reid Vapour Pressure (RVP).

Read the data set in and set the column called "Case" to be the index:
```python
distill = pd.read_csv("https://openmv.net/file/distillation-tower.csv")
```

In the [prior module](https://yint.org/pybasic14) you were asked to 

1. Calculate the correlation matrix of values and display that. Were you able to do so? 
2. Could you also visualize a scatter plot matrix of these values with the "kde" on the diagonal, squares for the markers and an alpha value of 0.8 for the points?
  
*Hint*: look at the documentation for `scatter_matrix` to see how to do this. You can look at the documentation inside Jupyter in several ways:
* ``help(scatter_matrix)``
* ``scatter_matrix?`` and then hit Ctrl-Enter.

In [ ]:
distill = pd.read_csv("https://openmv.net/file/distillation-tower.csv")
distill.shape

### Visualizing the correlation matrix

The correlation matrix of numbers can be tedious to read on a screen. You can easily visualize it though:

```python
import seaborn as sns
display(distill.corr());

# Let's visualize it instead, as a heat map.
sns.set(rc={'figure.figsize':(15, 15)})
sns.heatmap(distill.corr());

# This is not so attractive. Use a different colour map (cmap):
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(distill.corr(), cmap=cmap,  square=True, linewidths=0.2, cbar_kws={"shrink": 0.5});
```

### Visualizing the scatter plot matrix

We saw the scatter plots before, and the scatter plot matrix.

```python
from pandas.plotting import scatter_matrix
scatter_matrix(distill, alpha = 0.2, figsize=(15, 15),  diagonal = "kde");
```

The data set is quite big and takes some time to generate all the scatter plot combinations.

We can use every third row instead.
```python
print(distill.shape)
subset = distill.iloc[0::3, :]
subset.shape
```

The `.iloc` function accesses the data by `index` (the `i` in `iloc`) and for a given `loc`ation, so `iloc`= *index location*.

Some examples:
* `.iloc[0:10, :]` will return the \_\_\_ rows, and \_\_\_ columns
* `.iloc[20, 2:4]` will return only row \_\_\_, and columns \_\_\_
* `.iloc[0:10:2, :]` will return only rows with index \_\_\_; and \_\_\_ columns
* `.iloc[0::2, :]` will return \_\_\_ row; and \_\_\_ columns
* `.iloc[:, -1]` will return \_\_\_ rows of the \_\_\_ column

Now that you understand what `.iloc` is doing, you can understand why this code is faster, because it uses half the data set to create the scatter plot matrix:

```python
scatter_matrix(distill.iloc[0::2,:], alpha = 0.2, figsize=(15, 15),  diagonal = "kde");
```

Try some examples of `.iloc` yourself below:

Which 2 columns are the most correlated with the outcome variable called "VapourPressure"?

```python
distill.corr()
distill.corr().iloc[...] # <-- fill in some code to show only the last row of the correlation matrix
```

Let us build a regression model using that measurement of `InvTemp3`, the inverse temperature measured at position 3 in the distillation column, to predict the `VapourPressure`.

There are 253 measurements (rows) in the dataset. We will use the first 150 rows in the data set to build the model, and then use the remaining rows to test the model, to see how well we can predict vapour pressure. This is good statistical practice. Do not use all the data to build the prediction model; you will get an inflated sense of how well the model works.

Create the building data set from the first 150 rows
```python
build = distill.iloc[___]  <-- what goes here?
display(build)
```

Then the testing data from the rest:
```python
test = distill.iloc[___] <-- what goes here?
display(test)
```

Try it below, and use the `.shape` command to verify the `build`ing and `test`ing data have the correct shape.

First, we set up an `instance` of the linear regression model:
```python
mymodel = LinearRegression()
type(mymodel)
```

The `mymodel` is an object. It is an object of a linear regression model, but it is empty at the moment. We will provide it some training data, to build the model, in this way:

```python
mymodel = LinearRegression()
mymodel.fit(X, y)
```

but we have to tell it what is `X` and what is `y`. So we have a small detour...

We need numeric values for `X` and `y`. We can get those as follows. First a small detour...

```python
print(build["InvTemp3"])  # A Pandas Series (single column from the matrix of data)
print(build["InvTemp3"].values)  # NumPy vector of values, 
print(build["InvTemp3"].values.shape)  # Notice the vector is just a 1-D array of 30 values
print(build["InvTemp3"].values.reshape(-1, 1)) # Force them into a single column
print(build["InvTemp3"].values.reshape(-1, 1).shape)  # Now we have the right shape for scikit-learn
```

Scikit-learn requires the `X` data (the values used to predict `y`) to be a column vector or a matrix. Notice that a column vector is just a matrix with 1 column. This is because, you will see later, you can have 1 or more columns used to predict `y`. Therefore every input used to predict `y` must be in a column. Each row in the input matrix is one observation.

There is a shortcut to force the column to be extracted as a column:

```python
build[["InvTemp3"]].values
build[["InvTemp3"]].values.shape  # check what the shape is
```

So this will work to build your regression model:

```python
# A single column in matrix X (capital X indicates one or more input columns)
X = build[["InvTemp3"]].values
y = build["VapourPressure"].values
mymodel = LinearRegression()
mymodel.fit(X, y)
```

If you run this code and see no error messages, the model has been built. But it is not that exciting. 

So what can you do with this model? Use 
```python
dir(mymodel)
```
to ask Python what can be done. Note that the ``dir(...)`` function works on any object and is something that you will use regularly.

There are several interesting *methods* that you see there which we will get to use.

* `coef_`
* `intercept_`
* `predict`
* `score`

The first two, are as you might guess, the intercept of the model and the coefficient (slope).

```python
print(f"The intercept is {mymodel.intercept_} and the slope is = {mymodel.coef_}")
```

Now it is not so handy having all those decimal places. Python allows you to truncate them to the desired number:

```python
print(f"The intercept is {mymodel.intercept_:.5g} and the slope is = {mymodel.coef_}")
```

We have to be a bit more careful with the slope. It is an array (see the square brackets?): so we need to extract the first entry from that vector before displaying it:
```python
print(f"The intercept is {mymodel.intercept_:.5g} and the slope is = {mymodel.coef_[0]:.5g}")
```

Try this as well:
```python
print(f"The intercept is {mymodel.intercept_:.5f} and the slope is = {mymodel.coef_[0]:.5f}")
```
There is a subtle difference between the `f` and the `g` format specifiers.

## Detour: Visualization of the regression model

After building the regression model it is helpful to visualize it. The Seaborn library has a useful function to do so.

```python
import seaborn as sns
ax = sns.regplot(x="InvTemp3", y="VapourPressure", data=distill)
ax.grid(True)
```

Take a look at the documentation for the `regplot` function: https://seaborn.pydata.org/generated/seaborn.regplot.html

for more options, but the simple function above already does most of what you would expect:
* it draws a scatter plot of the raw data
* adds the regression line to the plot
* shows the confidence interval for the regression (the interval expected for the true but unknown slope)
* adds labels to the axes.

An "upgrade" you might be interested in, is the joint plot, which adds the histograms to the axes:

```python
sns.jointplot(x="InvTemp3", y="VapourPressure", data=distill, kind="reg");

# Or, show the kde=kernel density estimate
sns.jointplot(x="InvTemp3", y="VapourPressure", data=distill, kind="kde");

```

## Checking how well the regression model works: training data

Next, we would like to extract some idea of how the model performs. For that we can look at
* the predictions of the `build`ing data, 
* the predictions of the `test`ing data.

For the model building data:
```python
# Get the predicted values from the data used to build the model
X_build = build[["InvTemp3"]]
y_build = build["VapourPressure"].values

prediction_build = mymodel.predict(X_build)
errors_build = y_build - prediction_build # error = actual minus predicted

# There are several ways to see "how good" the model is, but the average 
# of the absolute values of the errors gives a good feeling. Smaller is better.
avg_absolute_error = pd.Series(errors_build).abs().mean()

```

1. Calculate this average absolute error below
2. Also calculate the standard deviation of the errors (another way to judge the model). Smaller is better.
3. Lastly, plot the prediction errors for the building data (first 200 rows) to see what time-based trends there are.

## Checking how well the regression model works: testing data

The above gives an idea of how the model works on the data used to build the model. But of course, the idea is to use the model in the future, on data not seen before. So let's test the model on the remaining rows:

```python
# Create the testing data set
test = distill.iloc[200:]
X_test = test[["InvTemp3"]].values
y_test = test["VapourPressure"].values
```

Then use the `predict(...)` function again, but on the testing data. Notice how simple scikit-learn makes this; just replace the input to the function with a different data frame:
```python
prediction_test = mymodel.predict(X_test)
errors_test = y_test - prediction_test
avg_absolute_error = pd.Series(errors_test).abs().mean()
avg_absolute_error, errors_test.std()
```

1. Calculate the average absolute error below, but for the model testing data
2. Calculate the standard deviation of the prediction errors (another way to judge the model). Smaller is better.
3. Lastly, plot the prediction errors for the testing data to see what time-based trends there are. Any concerns?

## Calculate the $R^2$ value

Using the `score` method, we can get the $R^2$ value. The function needs two inputs:
```python
mymodel.score(X_build, y_build)
```

and you will get a value that shows how the two variables are correlated. NOTE: the $R^2$ value is not a measure of goodness of the predictions. Only of the degree of correlation. High correlation is no guarantee of good prediction.

## Improving the model: adding extra predictors

A least squares model $$ y = b_0 + b_1x_1$$
with an intercept $b_0$ and a single slope of $b_1$ can be improved by adding a second, or more predictors: $$ y = b_0 + b_1x_1 + b_2x_2 + \ldots$$

This is called a multiple linear regression (MLR) model. We will try to improve our model by adding an extra predictor ``InvPressure1``:

```python

# Specify the predictors in a list:
predictors = ["InvTemp3", "InvPressure1"]

# Specify the training data:
X_build_MLR = build[predictors].values
y_build = build["VapourPressure"].values

# Fit the model
full_model = LinearRegression()
full_model.fit(X=X_build_MLR, y=y_build)

# Print some stats:
predict_MLR_build = full_model.predict(X_build_MLR)
errors_MLR_build = y_build - predict_MLR_build
avg_absolute_error_MLR_build = pd.Series(errors_MLR_build).abs().mean()
print(f"The average absolute error {avg_absolute_error_MLR_build:.3f}")
pd.Series(errors_MLR_build).plot(grid=True, title="Error = Actual - Predicted")
```

Notice the power here: you only have to change the first line to add new predictor. The rest of the code is the same as before (just more generic variable names).

## Challenge: checking the MLR model on the testing data

Try creating code for the testing data, which uses the 2 predictors.

* Extract the values for ``X_test_MLR`` and `y_test``.
* Use the ``full_model.predict(...)`` function
* Calculate the average absolute error and the standard deviation of the error.
* Plot the errors over time. Are they smaller than for the case where you had only 1 predictor?

In [ ]:
# IGNORE this. Execute this cell to load the notebook's style sheet.
from IPython.core.display import HTML
css_file = './images/style.css'
HTML(open(css_file, "r").read())